In [1]:
import pandas as pd
import requests
import numpy as np
import json
pd.options.mode.chained_assignment = None  # default='warn'

N=529
FD=10
TD=11

In [2]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()

In [3]:
urlf = []
rf = []
jsonf = []

for x in range(1,N):
    urlf.append('https://fantasy.premierleague.com/api/element-summary/{0}/'.format(x))
    rf.append(requests.get(urlf[x-1]))
    jsonf.append(rf[x-1].json())

In [4]:
elements_df = pd.DataFrame(json['elements'])
elements_type_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
events_df = pd.DataFrame(json['events'])
phases_df = pd.DataFrame(json['phases'])

fixtures = []

for y in range(1,N):
    fixtures.append(pd.DataFrame(jsonf[y-1]['fixtures']))
    
fpl=elements_df

In [5]:
fpl['position'] = fpl.element_type.map(elements_type_df.set_index('id').singular_name)
fpl['team'] = fpl.team_code.map(teams_df.set_index('code').name)
fpl['team_points'] = fpl.team_code.map(teams_df.set_index('code').points)
fpl['fixture_difficulty'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['3day'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['a1'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['a2'] = fpl.team_code.map(teams_df.set_index('code').strength)
fpl['a3'] = fpl.team_code.map(teams_df.set_index('code').strength)

In [6]:
fpl_slim=fpl[['id', 'code', 'first_name', 'second_name', 'team', 'position', 'now_cost', 'total_points', 
              'form', 'ict_index', 'fixture_difficulty','3day','a1','a2','a3','status']]
fpl_slim = fpl_slim.sort_values('id', ascending=True)

In [9]:
for z in range(1,N):
    fpl_slim.iloc[z-1,FD] = fixtures[z-1].iloc[0,1]
    fpl_slim.iloc[z-1,TD] = (fixtures[z-1].iloc[0,1]+fixtures[z-1].iloc[1,1]+fixtures[z-1].iloc[2,1])

In [10]:
A1 = fpl_slim['a1'].astype('float')
A2 = fpl_slim['a2'].astype('float')
A3 = fpl_slim['a3'].astype('float')

a=1
b=1
c=1
d=1
e=1
f=1


COST = fpl_slim['now_cost'].astype('float')
POINTS = fpl_slim['total_points'].astype('float')
FORM = fpl_slim['form'].astype('float')
ICT = fpl_slim['ict_index'].astype('float')
FIX = fpl_slim['fixture_difficulty'].astype('float')
TDAY = fpl_slim['3day'].astype('float')


for i in range(0,N):
    A1[i] =(POINTS[i]**a*ICT[i]**b*FORM[i]**c)/(COST[i]**d*FIX[i]**e*TDAY[i]**f)
    A2[i] =0
    A3[i] =0


fpl_slim['a1']=A1
fpl_slim['a2']=A2
fpl_slim['a3']=A3

In [11]:
fpl_slim1 = fpl_slim.sort_values('a1', ascending=False)

In [12]:
g=0
n=0
d=0
m=0
f=0
cost=0
points=0
M=4


fpl_fin = pd.DataFrame(columns=['id', 'code', 'first_name', 'second_name', 'team', 'position', 'now_cost', 'total_points', 
              'form', 'ict_index', 'fixture_difficulty','3day','a1','a2','a3','status'])


for j in range(1,N):

    if n < 16 and fpl_slim1.iloc[j-1,15] == 'a':
        
        if fpl_slim1.iloc[j-1,5] == 'Goalkeeper':
            g = g + 1
            if g < 3:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1

        if fpl_slim1.iloc[j-1,5] == 'Defender':
            d = d + 1
            if d < 6:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1
        
        if fpl_slim1.iloc[j-1,5] == 'Midfielder':
            m = m + 1
            if m < 6:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1
        
        if fpl_slim1.iloc[j-1,5] == 'Forward':
            f = f + 1
            if f < 4:
                fpl_fin = fpl_fin.append(fpl_slim1.loc[fpl_slim1['id'] == fpl_slim1.iloc[j-1,0]])
                n=n+1
                
    if n == 15:
        
        for k in range(1,16):
            cost = cost + fpl_fin.iloc[k-1,6]
            points = points + fpl_fin.iloc[k-1,7]
        print(cost/10)
        print(points)
        print('chk')
        if cost <= 1000:
            break
        if cost > 1000:
            
            cost = 0
            points=0
            n=n-1
            k=0
            fpl_fin.drop(fpl_fin.tail(1).index,inplace=True)    
            if fpl_slim1.iloc[j-1,5] == 'Goalkeeper':
                g = g - 1
            if fpl_slim1.iloc[j-1,5] == 'Defender':
                d = d - 1  
            if fpl_slim1.iloc[j-1,5] == 'Midfielder':
                m = m - 1        
            if fpl_slim1.iloc[j-1,5] == 'Forward':
                f = f - 1
    




95.6
186
chk


In [13]:
fpl_slim.to_csv('~/Desktop/fpl_data.csv')

In [14]:
fpl_fin.head(20)

,id,code,first_name,second_name,team,position,now_cost,total_points,form,ict_index,fixture_difficulty,3day,a1,a2,a3,status
281,254,118748,Mohamed,Salah,Liverpool,Midfielder,120,20,20.0,32.8,4,9,3.037037,0.0,0.0,a
372,485,83314,Jeff,Hendrick,Newcastle,Midfielder,50,14,14.0,10.3,2,9,2.243111,0.0,0.0,a
28,494,226597,Gabriel,Magalhães,Arsenal,Defender,50,15,15.0,8.5,2,10,1.912500,0.0,0.0,a
131,123,225796,Reece,James,Chelsea,Defender,50,14,14.0,11.8,4,8,1.445500,0.0,0.0,a
510,461,107613,Romain,Saïss,Wolves,Defender,50,15,15.0,10.2,4,8,1.434375,0.0,0.0,a
27,478,47431,Willian,Borges Da Silva,Arsenal,Midfielder,81,14,14.0,11.5,2,10,1.391358,0.0,0.0,a
170,155,101188,Lucas,Digne,Everton,Defender,60,12,12.0,6.1,2,6,1.220000,0.0,0.0,a
241,498,166477,Timothy,Castagne,Leicester,Defender,55,14,14.0,8.4,3,10,0.997818,0.0,0.0,a
114,105,85955,Jorge Luiz,Frello Filho,Chelsea,Midfielder,50,12,12.0,10.1,4,8,0.909000,0.0,0.0,a
225,224,101668,Jamie,Vardy,Leicester,Forward,100,13,13.0,11.8,3,10,0.664733,0.0,0.0,a
